## Set up

There is some set-up necessary to use OpenMP target offloading on Google Colab. These steps will need to be repeated for each notebook and each time a notebook is launched.  

### GCC 

By default the Colab VM comes with GCC 7.5.0

In [29]:
!gcc --version

gcc (Ubuntu 8.4.0-1ubuntu1~18.04) 8.4.0
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



For target offloading, we will need to install GCC 8 or later. Shouldn't take more than 10 seconds via `apt` 

In [30]:
!apt install gcc-8 g++-8 gfortran-8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-8 is already the newest version (8.4.0-1ubuntu1~18.04).
gcc-8 is already the newest version (8.4.0-1ubuntu1~18.04).
gfortran-8 is already the newest version (8.4.0-1ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


We then need to install the NVIDIA offloading package. 

In [31]:
!apt install gcc-8-offload-nvptx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc-8-offload-nvptx is already the newest version (8.4.0-1ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Now make GCC 8 the default.

In [32]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 80 --slave /usr/bin/g++ g++ /usr/bin/g++-8

Verify the default is set to version 8. 

In [33]:
!gcc --version 

gcc (Ubuntu 8.4.0-1ubuntu1~18.04) 8.4.0
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.




### NVIDIA driver 

By default Colab uses NVIDIA 11 (last checked July, 22). NVIDIA 11 has deprecated CUDA architecture `SM_30`. Unfortunately, this breaks builds GCC OpenMP offloading because it does not provide a flag for picking a particular architecture. 

The simplest workaround is to rollback to CUDA 10

In [34]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [35]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.0 /usr/local/cuda

Check to make sure we have a GPU Runtime. 

In [36]:
!nvidia-smi

Thu Jul 21 19:27:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Analysis

### Quicksort Hybrid



In [37]:
%%writefile mergesort_hybrid.c
/* 
 * Recursive parallel-hybrid implementation of MergeSort (not optimized!)
 * This code is to be used as examples in module [B1] Hybrid Algorithm
 *
 * parts of the code borrow from
 *   : https://stackoverflow.com/questions/13811114/parallel-merge-sort-in-openmp
 * 
 * @author: Apan Qasem <apan@txstate.edu>
 * @date: 04/04/21 
 * 
 * @update: 07/18/22 
 */


#include <string.h>
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>
#include <sys/time.h>

#define VAL_RANGE 1024
#define ELEMENTS_TO_VERIFY 5
#define PAR_THRESHOLD 100


/*
 *  retrieve time in seconds from getitimeofday()
 */
double timer() {
  struct timeval tp;
  struct timezone tzp;
  int i;

  i = gettimeofday(&tp,&tzp);
  return ( (double) tp.tv_sec + (double) tp.tv_usec * 1.e-6 );
}

/* 
 * display array contents 
 */
void display(double values[], long long N) {
  for (int i = 0; i < N; i++)
    fprintf(stdout, "%3.4f ", values[i]);
  fprintf(stdout, "\n");
}


#pragma omp declare target 
void merge(double *values, int n, double *aux) {
   int i = 0;
   int j = n/2;
   int aux_index = 0;

   while (i < n/2 && j < n) {
     if (values[i] < values[j]) {
       aux[aux_index] = values[i];
       aux_index++; i++;
     } else {
       aux[aux_index] = values[j];
       aux_index++; j++;
     }
   }

   while (i < n/2) { 
      aux[aux_index] = values[i];
      aux_index++;
      i++;
   }
   while (j < n) { 
      aux[aux_index] = values[j];
      aux_index++;
      j++;
   }
   memcpy(values, aux, n * sizeof(double));
} 
#pragma omp end declare target 

#pragma omp declare target 
void merge_sort_gpu(double *values, int n, double *aux) {
   if (n < 2) return;

   unsigned int ub = n/2;
   merge_sort_gpu(values, n/2, aux);
   merge_sort_gpu(values + (n/2), n - (n/2), aux + n/2);
   merge(values, n, aux);
}
#pragma omp end declare target 

void merge_sort_cpu(double *values, int n, double *aux) {
   if (n < 2)
     return;

   #pragma omp task shared(values) if (n > PAR_THRESHOLD)
   merge_sort_cpu(values, n/2, aux);

   #pragma omp task shared(values) if (n > PAR_THRESHOLD)
   merge_sort_cpu(values + (n/2), n - (n/2), aux + n/2);

   #pragma omp taskwait
   merge(values, n, aux);
}

void merge_sort_driver(double *values, int n, double *aux) {
   if (n < 2)
     return;

   unsigned int ub = n/2;
   #pragma omp target map(tofrom:values[0:ub]) map(tofrom:aux[0:ub])
   {
     merge_sort_gpu(values, n/2, aux);
   }
   #pragma omp task shared(values) if (n > PAR_THRESHOLD)
   merge_sort_cpu(values + (n/2), n - (n/2), aux + n/2);

   #pragma omp taskwait
   merge(values, n, aux);
}


int main(int argc, char *argv[]) {

  if (argc < 3) {
    printf("usage: \n");
    printf("       ./mergesort N threads\n");
    printf("       N = input size\n"); 
    printf("       t = number of OpenMP threads\n"); 
    exit(0);
  }
  
  long long N = atoi(argv[1]);
  unsigned threads = atoi(argv[2]);
  
  omp_set_num_threads(threads);
  
  double *values = (double *) malloc(sizeof(double) * N);
  double *aux = (double *) malloc(sizeof(double) * N);

  for (int i = 0; i < N; i++) 
    values[i] = rand() / (double) (RAND_MAX/VAL_RANGE);


  omp_set_dynamic(0);   

  double t0 = timer();
  #pragma omp parallel
  {
    #pragma omp single
    merge_sort_driver(values, N, aux);
  }   
  t0 = (timer() - t0) * 1.e3;
  
  fprintf(stdout, "Sorted values [0..%d]: ", ELEMENTS_TO_VERIFY - 1);
  display(values, ELEMENTS_TO_VERIFY);
  fprintf(stdout, "Execution time = %3.2f ms\n", t0); 
  
  
  return 0; 
}

Overwriting mergesort_hybrid.c


We can compile the code with the following

In [40]:
!gcc -o mergesort_hybrid mergesort_hybrid.c -fno-stack-protector -foffload=nvptx-none -fopenmp 

In [ ]:
!nvprof -u ms ./mergesort_hybrid 10000 1 

### Quicksort CPU parallel

In [ ]:
%%writefile mergesort_par.c
/* 
 * Recursive parallel implementation of MergeSort (not optimized!)
 * This code is to be used in conjunction with exercises in module [B1] Hybrid Algorithm
 *
 * parts of the code borrow from
 *   : https://stackoverflow.com/questions/13811114/parallel-merge-sort-in-openmp
 * 
 * @author: Apan Qasem <apan@txstate.edu>
 * @date: 04/04/21 
 * 
 * @update: 07/18/22 
 */


#include <string.h>
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>
#include <sys/time.h>

#define VAL_RANGE 1024
#define ELEMENTS_TO_VERIFY 5
#define PAR_THRESHOLD 100


/*
 *  retrieve time in seconds from getitimeofday()
 */
double timer() {
  struct timeval tp;
  struct timezone tzp;
  int i;

  i = gettimeofday(&tp,&tzp);
  return ( (double) tp.tv_sec + (double) tp.tv_usec * 1.e-6 );
}

/* 
 * display array contents 
 */
void display(double values[], long long N) {
  for (int i = 0; i < N; i++)
    fprintf(stdout, "%3.4f ", values[i]);
  fprintf(stdout, "\n");
}


void merge(double *values, int n, double *aux) {
   int i = 0;
   int j = n/2;
   int aux_index = 0;

   while (i < n/2 && j < n) {
     if (values[i] < values[j]) {
       aux[aux_index] = values[i];
       aux_index++; i++;
     } else {
       aux[aux_index] = values[j];
       aux_index++; j++;
     }
   }

   while (i < n/2) { 
      aux[aux_index] = values[i];
      aux_index++;
      i++;
   }
   while (j < n) { 
      aux[aux_index] = values[j];
      aux_index++;
      j++;
   }
   memcpy(values, aux, n * sizeof(double));
} 

void merge_sort(double *values, int n, double *aux) {
   if (n < 2) return;

   #pragma omp task shared(values) if (n > PAR_THRESHOLD)
   merge_sort(values, n/2, aux);

   #pragma omp task shared(values) if (n > PAR_THRESHOLD)
   merge_sort(values + (n/2), n - (n/2), aux + n/2);

   #pragma omp taskwait
   merge(values, n, aux);
}

int main(int argc, char *argv[]) {

  if (argc < 3) {
    printf("usage: \n");
    printf("       ./mergesort N threads\n");
    printf("       N = input size\n"); 
    printf("       t = number of OpenMP threads\n"); 
    exit(0);
  }
  
  long long N = atoi(argv[1]);
  unsigned threads = atoi(argv[2]);
  
  omp_set_num_threads(threads);
  
  double *values = (double *) malloc(sizeof(double) * N);
  double *aux = (double *) malloc(sizeof(double) * N);

  for (int i = 0; i < N; i++) 
    values[i] = rand() / (double) (RAND_MAX/VAL_RANGE);

  omp_set_dynamic(0);   
  
  double t0 = timer();
  #pragma omp parallel
  {
    #pragma omp single
    merge_sort(values, N, aux);
  }   
  t0 = (timer() - t0) * 1.e3;
  
  fprintf(stdout, "Sorted values [0..%d]: ", ELEMENTS_TO_VERIFY - 1);
  display(values, ELEMENTS_TO_VERIFY);
  fprintf(stdout, "Execution time = %3.2f ms\n", t0); 
  
  
  return 0; 
}

 

In [ ]:
!gcc -o mergesort_par mergesort_par.c -fopenmp 

In [ ]:
!./mergesort_par 10000 1

In [ ]:
!./mergesort_par 10000 4